https://github.com/nomomon/Amazon-Product-Description/tree/main

In [ ]:
!git clone https://github.com/nomomon/Amazon-Product-Description.git

In [ ]:
%cd /kaggle/working/Amazon-Product-Description

Prepare Dataset

https://jmcauley.ucsd.edu/data/amazon_v2/index.html

In [ ]:
import urllib.request

def download_file(category): 
    metadata_url = "https://jmcauley.ucsd.edu/data/amazon_v2/metaFiles/meta_"+ category +".json.gz" 
    urllib.request.urlretrieve(metadata_url, './data.json.gz')

In [ ]:
import gzip
import json

def read_file():
    counter = 0
    data = []
    with gzip.open('data.json.gz') as f:
        for l in f:
            if counter < 3000:
                data.append(json.loads(l.strip()))
                counter+=1
        
    return data

In [ ]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [ ]:
def clean(item):
    if type(item["description"]).__name__ == "str":
        item["description"] = [item["description"]]
    if type(item["category"]).__name__ == "str":
        item["category"] = [item["category"]]
    if type(item["feature"]).__name__ == "str":
        item["feature"] = [item["feature"]]
    return item

In [ ]:
import re

def cleanhtml(raw_html):
  cleanr = re.compile('<.*?>')
  cleantext = re.sub(cleanr, '', raw_html)
  return cleantext

In [ ]:
def convert_to_prompt(item):
    title = item["title"]
    categories = ", ".join(item["category"])
    feature = ", ".join(item["feature"])
    description = ". ".join(item["description"])

    description = cleanhtml(description)
    categories = cleanhtml(categories)
    feature = cleanhtml(feature)


    if len(categories) > 10 and len(title) > 10 and len(description.split(' ')) >= 10 and len(description.split(' ')) <= 60 and len(feature) > 10:
        prompt = f'''Categories: {categories}
Title: {title}
Description: {description}
Features: {feature}
<|endoftext|>'''
    else:
        prompt = ""

    return prompt

Put to File

In [ ]:
def put_to_file(category, text):
    f = open(f"dataset/{category}.txt", "a+")
    f.write(text)
    
    f.close()


Merge Everything

In [ ]:
categories = [
    "Electronics",
    "Home_and_Kitchen",
    "Toys_and_Games",
]

done = []

In [ ]:
import os, shutil
folder = 'dataset'
for filename in os.listdir(folder):
    file_path = os.path.join(folder, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))

In [ ]:
def doAll():
    for category in categories:
        if category not in done:    
            print(f"Doing {category} ...")

            subCat = category.replace(" ", "_")
            download_file(subCat)
            print("     Downloaded")
            for item in read_file():
                put_to_file(subCat, convert_to_prompt(clean(item)))
            print("     Placed in a file")

            done.append(category)

doAll()

Split Dataset Train dan Dataset Test

In [ ]:
%mkdir dataset_train

In [ ]:
%mkdir dataset_test

In [ ]:
def parse_dataset(file_path, separator):
    with open(file_path, 'r', encoding='utf-8') as file:
        # Gunakan string pemisah untuk membagi konten file
        entries = file.read().strip().split(separator)

    parsed_data = []
    for entry in entries:
        lines = entry.split('\n')
        entry_dict = {}
        for line in lines:
            if ': ' in line:
                key, value = line.split(': ', 1)
                entry_dict[key] = value
        parsed_data.append(entry_dict)

    return parsed_data

def split_dataset(dataset, first_split_size):
    return dataset[:first_split_size], dataset[first_split_size:500]

def write_dataset_to_file(dataset, file_path):
    with open(file_path, 'w', encoding='utf-8') as file:
        for entry in dataset:
            for key, value in entry.items():
                file.write(f"{key}: {value}\n")
            file.write("<|endoftext|>\n")  # Menambahkan pemisah antar entri
            
for category in categories:
    subCat = category.replace(" ", "_")
    file_path = f'dataset/{subCat}.txt'
    separator = '<|endoftext|>'
    dataset = parse_dataset(file_path, separator)
    dataset_200, dataset_100 = split_dataset(dataset, 400)
    write_dataset_to_file(dataset_200, f'dataset_train/{subCat}.txt')
    write_dataset_to_file(dataset_100, f'dataset_test/{subCat}.txt')

Merge dataset with copy *.txt Merge{Train/Test}Dataset.txt

Fine Tuning GPT 2

In [1]:
!pip install happytransformer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 9.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.1/330.1 kB 23.7 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.16.4
    Uninstalling huggingface-hub-0.16.4:
      Successfully uninstalled huggingface-hub-0.16.4
  Attempting uninstall: datasets
    Found existing installation: datasets 2.1.0
    Uninstalling datasets-2.1.0:
      Successfully uninstalled datasets-2.1.0


In [ ]:
from happytransformer import HappyGeneration

In [ ]:
happy_gen = HappyGeneration("GPT2", "gpt2")

In [ ]:
from happytransformer import GENTrainArgs

In [ ]:
train_args = GENTrainArgs(learning_rate = 3e-6, num_train_epochs = 1)
happy_gen.train("/kaggle/input/merge-train-dataset-6-0/MergeTrainDataset.txt", args = train_args)

In [ ]:
happy_gen.save("/kaggle/working/GPT2_Amazon/")

Test Model

In [2]:
!pip install Rouge

In [3]:
import nltk
from nltk.translate.bleu_score import sentence_bleu
nltk.download('punkt')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
from happytransformer import HappyGeneration

happy_gen = HappyGeneration(load_path="/kaggle/input/gpt-2-model-skripsi-6-0")

You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embedding dimension will be 50257. This might induce some performance reduction as *Tensor Cores* will not be available. For more details about this, or help on choosing the correct value for resizing, refer to this guide: https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc


In [ ]:
from torchinfo import summary
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained('gpt2')
summary(model)

In [5]:
from happytransformer import GENSettings

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [7]:
from rouge import Rouge

def parse_dataset(file_path, separator):
    with open(file_path, 'r', encoding='utf-8') as file:
        # Gunakan string pemisah untuk membagi konten file
        entries = file.read().strip().split(separator)

    parsed_data = []
    for entry in entries:
        lines = entry.split('\n')
        entry_dict = {}
        for line in lines:
            if ': ' in line:
                key, value = line.split(': ', 1)
                entry_dict[key] = value
        parsed_data.append(entry_dict)

    return parsed_data

dataset = parse_dataset('/kaggle/input/merge-test-dataset-6-0/MergeTestDataset.txt', '<|endoftext|>')

def calculate_rouge_l_f_scores(model_summaries, reference_summaries):
    rouge = Rouge()
    scores = rouge.get_scores(model_summaries, reference_summaries)
    return [score['rouge-l']['f'] for score in scores]

def calculate_bleu_scores(model_summaries, reference_summaries):
    bleu_scores = []
    for model, reference in zip(model_summaries, reference_summaries):
        # Tokenize the sentences
        model_tokens = nltk.word_tokenize(model)
        reference_tokens = [nltk.word_tokenize(reference)]
        
        # Calculate BLEU score
        score = sentence_bleu(reference_tokens, model_tokens)
        bleu_scores.append(score)
    
    return bleu_scores

def test_model():
    model_summaries = []
    reference_summaries = []
    model_summaries_2 = []
    reference_summaries_2 = []
    model_summaries_3 = []
    reference_summaries_3 = []

    for data in dataset:
        if data == {}: 
            continue
        if len(data['Description'].split(' ')) < 10:
            continue
    
        descs = data['Description'].split(' ')
    
        text = f"""Categories: {data['Categories']}
        Title: {data['Title']}
        Features: {data['Features']}
        Description: {descs[0]} {descs[1]}"""
    
        result = happy_gen.generate_text(text, args=gen_args)
        output = result.text

        if data['Categories'].split(',')[0] == "Electronics":
            model_summaries.append(output) # Replace this with your model-generated summaries
            reference_summaries.append(data['Description']) # Replace this with your reference summaries
        if data['Categories'].split(',')[0] == "Home & Kitchen":
            model_summaries_2.append(output) # Replace this with your model-generated summaries
            reference_summaries_2.append(data['Description']) # Replace this with your reference summaries
        if data['Categories'].split(',')[0] == "Toys & Games":
            model_summaries_3.append(output) # Replace this with your model-generated summaries
            reference_summaries_3.append(data['Description']) # Replace this with your reference summaries
    
    f_scores = calculate_rouge_l_f_scores(model_summaries, reference_summaries)
    f_scores_2 = calculate_rouge_l_f_scores(model_summaries_2, reference_summaries_2)
    f_scores_3 = calculate_rouge_l_f_scores(model_summaries_3, reference_summaries_3)

    bleu_scores = calculate_bleu_scores(model_summaries, reference_summaries)
    bleu_scores_2 = calculate_bleu_scores(model_summaries_2, reference_summaries_2)
    bleu_scores_3 = calculate_bleu_scores(model_summaries_3, reference_summaries_3)
    
    return f_scores, f_scores_2, f_scores_3, bleu_scores, bleu_scores_2, bleu_scores_3

In [8]:
def evaluation_metric():
    rouge_avg = []
    rouge_avg.append(np.mean(f_scores))
    rouge_avg.append(np.mean(f_scores_2))
    rouge_avg.append(np.mean(f_scores_3))

    bleu_avg = []
    bleu_avg.append(np.mean(bleu_scores))
    bleu_avg.append(np.mean(bleu_scores_2))
    bleu_avg.append(np.mean(bleu_scores_3))

    evaluation_score = {
        "Rouge Average": rouge_avg,
        "Bleu Average": bleu_avg
    }

    df = pd.DataFrame(evaluation_score, index = ["Electronics","Home and Kitchen","Toys and Games",])
    return df

In [9]:
def std():
    rouge_std = []
    rouge_std.append(np.std(f_scores))
    rouge_std.append(np.std(f_scores_2))
    rouge_std.append(np.std(f_scores_3))

    bleu_std = []
    bleu_std.append(np.std(bleu_scores))
    bleu_std.append(np.std(bleu_scores_2))
    bleu_std.append(np.std(bleu_scores_3))

    evaluation_score = {
        "Rouge Standard Deviation": rouge_std,
        "Bleu Standard Deviation": bleu_std
    }

    df = pd.DataFrame(evaluation_score, index = ["Electronics","Home and Kitchen","Toys and Games",])
    return df

Experiment Temperature

In [10]:
min_length =  50
max_length = 100 
do_sample = True
early_stopping = True
num_beams = 1 
temperature = 1.4
top_k = 50
top_p = 0.7
no_repeat_ngram_size = 2

gen_args = GENSettings(min_length, max_length, do_sample, early_stopping, num_beams, temperature, top_k, no_repeat_ngram_size, top_p)

In [11]:
f_scores, f_scores_2, f_scores_3, bleu_scores, bleu_scores_2, bleu_scores_3 = test_model()

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:399: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 f

In [12]:
df = evaluation_metric()
df

,Rouge Average,Bleu Average
Electronics,0.093304,0.499474
Home and Kitchen,0.089165,0.435429
Toys and Games,0.100482,0.421528


In [13]:
df = std()
df

,Rouge Standard Deviation,Bleu Standard Deviation
Electronics,0.042961,0.153742
Home and Kitchen,0.055468,0.165130
Toys and Games,0.052559,0.183605


In [14]:
min_length =  50
max_length = 100 
do_sample = True
early_stopping = True
num_beams = 1 
temperature = 0.7
top_k = 50
top_p = 0.7
no_repeat_ngram_size = 2

gen_args = GENSettings(min_length, max_length, do_sample, early_stopping, num_beams, temperature, top_k, no_repeat_ngram_size, top_p)

In [15]:
f_scores, f_scores_2, f_scores_3, bleu_scores, bleu_scores_2, bleu_scores_3 = test_model()

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:399: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 f

In [16]:
df = evaluation_metric()
df

,Rouge Average,Bleu Average
Electronics,0.108874,0.475601
Home and Kitchen,0.107885,0.393437
Toys and Games,0.128802,0.350456


In [17]:
df = std()
df

,Rouge Standard Deviation,Bleu Standard Deviation
Electronics,0.054692,0.175443
Home and Kitchen,0.059221,0.192258
Toys and Games,0.056282,0.192367


In [ ]:
f_scores, f_scores_2, f_scores_3, bleu_scores, bleu_scores_2, bleu_scores_3 = test_model()

In [18]:
min_length =  50
max_length = 100 
do_sample = True
early_stopping = True
num_beams = 1 
temperature = 0.1
top_k = 50
top_p = 0.7
no_repeat_ngram_size = 2

gen_args = GENSettings(min_length, max_length, do_sample, early_stopping, num_beams, temperature, top_k, no_repeat_ngram_size, top_p)

In [19]:
f_scores, f_scores_2, f_scores_3, bleu_scores, bleu_scores_2, bleu_scores_3 = test_model()

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:399: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 f

In [20]:
df = evaluation_metric()
df

,Rouge Average,Bleu Average
Electronics,0.118638,0.451134
Home and Kitchen,0.109428,0.390020
Toys and Games,0.141092,0.362344


In [21]:
df = std()
df

,Rouge Standard Deviation,Bleu Standard Deviation
Electronics,0.052407,0.199177
Home and Kitchen,0.072428,0.192505
Toys and Games,0.064422,0.189621


Experiment Top_K

In [22]:
min_length =  50
max_length = 100 
do_sample = True
early_stopping = True
num_beams = 1 
temperature = 0.7
top_k = 1
top_p = 0.7
no_repeat_ngram_size = 2

gen_args = GENSettings(min_length, max_length, do_sample, early_stopping, num_beams, temperature, top_k, no_repeat_ngram_size, top_p)

In [23]:
f_scores, f_scores_2, f_scores_3, bleu_scores, bleu_scores_2, bleu_scores_3 = test_model()

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:399: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 f

In [24]:
df = evaluation_metric()
df

,Rouge Average,Bleu Average
Electronics,0.118801,0.460337
Home and Kitchen,0.110987,0.401137
Toys and Games,0.140693,0.351904


In [25]:
df = std()
df

,Rouge Standard Deviation,Bleu Standard Deviation
Electronics,0.049063,0.188444
Home and Kitchen,0.075414,0.196262
Toys and Games,0.055621,0.178407


In [26]:
min_length =  50
max_length = 100 
do_sample = True
early_stopping = True
num_beams = 1 
temperature = 0.7
top_k = 100
top_p = 0.7
no_repeat_ngram_size = 2

gen_args = GENSettings(min_length, max_length, do_sample, early_stopping, num_beams, temperature, top_k, no_repeat_ngram_size, top_p)

In [27]:
f_scores, f_scores_2, f_scores_3, bleu_scores, bleu_scores_2, bleu_scores_3 = test_model()

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:399: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 f

In [28]:
df = evaluation_metric()
df

,Rouge Average,Bleu Average
Electronics,0.109799,0.477239
Home and Kitchen,0.100407,0.416441
Toys and Games,0.125323,0.378505


In [29]:
df = std()
df

,Rouge Standard Deviation,Bleu Standard Deviation
Electronics,0.055630,0.172220
Home and Kitchen,0.058968,0.172696
Toys and Games,0.056618,0.183434
